In [1]:
import numpy as np
import pandas as pd
import sqlite3

In [2]:
df = pd.read_csv(r'C:\Users\Gaspar\Desktop\Coctails_db\all_drinks.csv', delimiter = ",")

In [3]:
def to_lower(text):
    return text.lower()


df['strGlass'] = df['strGlass'].apply(to_lower)
df['strCategory'] = df['strCategory'].apply(to_lower)

In [4]:
conn = sqlite3.connect('cocktails_database.db')

In [5]:
df_glass = pd.read_sql_query("SELECT * FROM glass", conn)

In [6]:
df = pd.merge(df, df_glass, left_on='strGlass', right_on='Glass')

In [7]:
df = df.rename(columns={'ID': 'id_glass'})

In [8]:
df_categories = pd.read_sql_query("SELECT * FROM category", conn)

In [9]:
df = pd.merge(df, df_categories, left_on='strCategory', right_on='Category')

In [10]:
df = df.rename(columns={'ID': 'id_category'})

In [11]:
df = df.sort_values(by = 'strDrink')

In [12]:
df_temp = pd.DataFrame()

In [13]:
df_temp = df[['strDrink', 'strInstructions', 'id_glass', 'id_category']]

In [14]:
df_temp = df_temp.rename(columns={'strDrink': 'Cocktail', 'strInstructions': 'Instruction'})

In [15]:
ids = range(1, len(df_temp) + 1)

In [16]:
data_dict = {'ID': ids, 'Cocktails': df_temp['Cocktail'], 'Instruction': df_temp['Instruction'],
             'id_glass': df_temp['id_glass'],'id_category': df_temp['id_category']}
df_cocktail = pd.DataFrame(data_dict)

In [17]:
df_cocktail.head()

,ID,Cocktails,Instruction,id_glass,id_category
0,1,'57 Chevy with a White License Plate,1. Fill a rocks glass with ice 2.add white cre...,1,1
375,2,1-900-FUK-MEUP,Shake ingredients in a mixing tin filled with ...,2,2
476,3,110 in the shade,Drop shooter in glass. Fill with beer,3,3
428,4,151 Florida Bushwacker,Combine all ingredients. Blend until smooth. G...,4,4
20,5,155 Belmont,Blend with ice. Serve in a wine glass. Garnish...,5,1


In [18]:
df_cocktail.to_sql('cocktail', conn, index=False, if_exists='replace')

546

In [19]:
conn.close()